<a href="https://colab.research.google.com/github/BF667/TSRVC-ipynb/blob/main/terastudio-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TeraStudio RVC**


A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/TEywTVdK) — [GitHub](https://github.com/terastudio-org/TeraStudio-RVC)
<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas:
* PhamHuynhAnh16 (Base Project)


#### **Disclaimer**
By using TeraStudio RVC, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while TeraStudio RVC disclaims liability.

In [ ]:
#@title **🌏 Setting**
import os

from ipywidgets import Button
from IPython.display import clear_output

print("👩🏻‍💻 Cài đặt...")

!git clone https://github.com/terastudio-org/TeraStudio-RVC.git /content/main_core > /dev/null 2>&1
!apt-get -y install libportaudio2 -qq > /dev/null 2>&1


app_dir = "content/main_core"

!pip install uv > /dev/null 2>&1
!uv pip install -r /content/main_core/requirements.txt --no-cache-dir -q > /dev/null 2>&1

#@markdown **💻 Installation will take about 2 minutes to complete!**
%cd app_dir
clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **📱 Launch User Interface**
import os
import json

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
configs_json = os.path.join("main", "configs", "config.json")

def change_language(lang):
    configs = json.load(open(configs_json, "r"))

    if lang != configs["language"]:
        configs["language"] = lang

        with open(configs_json, "w") as f:
            json.dump(configs, f, indent=4)

%cd /content/Vietnamese_RVC
#@markdown **To experience all features, use the interface :) For a simpler approach, do not use the interface.**
language = "en-US" #@param ["vi-VN", "en-US"]
#@markdown **If you know how, you can use charts to check for overfitting 👍**
use_charts = False #@param {type:"boolean"}

if use_charts:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

change_language(language)

if not os.path.exists(f"/content/{app_dir}/assets/models/predictors/rmvpe.pt"):
    !wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/predictors/rmvpe.pt -O /content/Vietnamese_RVC/assets/models/predictors/rmvpe.pt > /dev/null 2>&1

if not os.path.exists(f"/content/{app_dir}/assets/models/embedders/hubert_base.pt"):
    !wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/embedders/fairseq/hubert_base.pt -O /content/Vietnamese_RVC/assets/models/embedders/hubert_base.pt > /dev/null 2>&1

!python main/app/app.py --share --client

# **Further customization 🧰**

In [ ]:
#@title **Connect or Disconnect to Drive ☁**
import os

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output, display

if os.path.exists("/content/drive"):
    print("🔗 Disconnecting from drive...")
    try:
        drive.flush_and_unmount()
        clear_output()
        display(Button(description="\u2714 Completed!", button_style="success"))
    except Exception as e:
        raise ValueError(f'An error occurred during drive disconnection: {e}')
else:
    print('🔗 Connecting to drive...')
    try:
        drive.mount('/content/drive')
        clear_output()
        display(Button(description="\u2714 Completed!", button_style="success"))
    except Exception as e:
        raise ValueError(f'An error occurred during drive connection: {e}')

In [ ]:
#@title **Start or Stop Backup 🛠**
import os
import time
import threading
import subprocess

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output

logs_folder, weights_folder, audios_folder = '/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **If no box is checked, the connection will be stopped for that part**
start_model_backup = False #@param {"type":"boolean"}
start_audio_backup = False #@param {"type":"boolean"}
#@markdown **Synchronization will sync the backup folders; if a file is deleted from the source folder, it will also be deleted from the backup folder**
synchronize_folders = False #@param {"type":"boolean"}


class Channel:
    def __init__(self, source, destination, sync_deletions=False, every=60, exclude = None):
        self.source = source
        self.destination = destination
        self.event = threading.Event()
        self.syncing_thread = threading.Thread(target=self._sync, args=())
        self.sync_deletions = sync_deletions
        self.every = every

        if not exclude: exclude = []
        if isinstance(exclude, str): exclude = [exclude]

        self.exclude = exclude
        self.command = ['rsync', '-aP']

    def alive(self):
        if self.syncing_thread.is_alive(): return True
        else: return False

    def _sync(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')

        while not self.event.is_set():
            subprocess.run(command)
            time.sleep(self.every)

    def copy(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')
        subprocess.run(command)

        return True

    def start(self):
        if self.syncing_thread.is_alive():
            self.event.set()
            self.syncing_thread.join()

        if self.event.is_set(): self.event.clear()
        if self.syncing_thread._started.is_set(): self.syncing_thread = threading.Thread(target=self._sync, args=())

        self.syncing_thread.start()
        return self.alive()

    def stop(self):
        if self.alive():
            self.event.set()
            self.syncing_thread.join()

            while self.alive():
                if not self.alive(): break

        return not self.alive()

if not "logs_backup" in locals(): logs_backup = Channel("/content/Vietnamese_RVC/assets/logs", logs_folder, sync_deletions=synchronize_folders, every=40, exclude="mute")
if not "weights_backup" in locals(): weights_backup = Channel("/content/Vietnamese_RVC/assets/weights", weights_folder, sync_deletions=synchronize_folders, every=40)
if not "audio_backup" in locals(): audio_backup = Channel("/content/Vietnamese_RVC/audios", audios_folder, sync_deletions=synchronize_folders, every=40)

logs_backup.stop(); weights_backup.stop(); audio_backup.stop()

if os.path.exists('/content/drive/MyDrive'):
    if start_model_backup:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if start_audio_backup:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()
else:
    try:
        drive.mount('/content/drive')
    except Exception as e:
        raise ValueError(f'An error occurred during drive connection: {e}')

    if start_model_backup:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if start_audio_backup:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()

clear_output()
Button(description="\u2714 Completed!", button_style="success")

In [ ]:
#@title **♻️ Start Garbage Collection**
import os
import time
import threading

from ipywidgets import Button
from google.colab import auth, drive
from IPython.display import clear_output

try:
    from googleapiclient.discovery import build
except:
    os.system("pip install google-api-python-client")
    from googleapiclient.discovery import build

class Clean:
    def __init__(self, every=60):
        self.service = build('drive', 'v3')
        self.every = every
        self.trash_cleanup_thread = None

    def delete(self):
        page_token = None

        while 1:
            response = self.service.files().list(q="trashed=true", spaces='drive', fields="nextPageToken, files(id, name)", pageToken=page_token).execute()

            for file in response.get('files', []):
                if file['name'].startswith("G_") and file['name'].endswith(".pth") or file['name'].startswith("D_") and file['name'].endswith(".pth"):
                    try:
                        self.service.files().delete(fileId=file['id']).execute()
                    except Exception as e:
                        raise RuntimeError(e)

            page_token = response.get('nextPageToken', None)
            if page_token is None: break

    def clean(self):
        while 1:
            self.delete()
            time.sleep(self.every)

    def start(self):
        self.trash_cleanup_thread = threading.Thread(target=self.clean)
        self.trash_cleanup_thread.daemon = True
        self.trash_cleanup_thread.start()

    def stop(self):
        if self.trash_cleanup_thread: self.trash_cleanup_thread.join()

%cd /content/Vietnamese_RVC

#@markdown **Use during training to clean up D_, G_ files in Google Drive trash**
start_garbage_collection = False #@param {"type":"boolean"}

if start_garbage_collection:
    if os.path.exists('/content/drive/MyDrive'):
        auth.authenticate_user()
    else:
        try:
            drive.mount('/content/drive')
        except Exception as e:
            raise ValueError(f'An error occurred during drive connection: {e}')

        auth.authenticate_user()
    Clean(every=40).start()
else: Clean().stop()

clear_output()
Button(description="\u2714 Completed!", button_style="success")

In [ ]:
#@title **Load Backup Data from Drive 📂**
import os
import shutil
from ipywidgets import Button
from IPython.display import clear_output, display

logs_folder, weights_folder, audios_folder ='/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Load trained models to continue training**
load_models = False # @param {"type":"boolean"}
#@markdown **Load backed up audios to continue using**
load_audios = False # @param {"type":"boolean"}

if os.path.exists("/content/drive/MyDrive"):
  if load_models:
    if len(os.listdir(logs_folder)) < 1 or len(os.listdir(weights_folder)) < 1: print("No Data Found")
    else:
      if os.path.exists("/content/drive/MyDrive/model"):
        shutil.copytree(logs_folder, "/content/main_core/assets/logs", dirs_exist_ok=True)
        shutil.copytree(weights_folder, "/content/main_core/assets/weights", dirs_exist_ok=True)

        clear_output()
      else: print("No model data found")
  if load_audios:
    if len(os.listdir(audios_folder)) < 1: print("No Data Found")
    else:
      if os.path.exists("/content/drive/MyDrive/audios"):
        shutil.copytree(audios_folder, "/content/main_core/audios", dirs_exist_ok=True)
        clear_output()
      else: print("No audio data found")
else: print("Google Drive is not connected")

display(Button(description="\u2714 Completed!", button_style="success"))

# **Download 📩**

In [ ]:
#@title **🔎 Search for Models**
import json
import codecs
import requests

from bs4 import BeautifulSoup

def fetch_models_data(search):
    all_table_data = []
    page = 1

    while 1:
        try:
            response = requests.post(url=codecs.decode("uggcf://ibvpr-zbqryf.pbz/srgpu_qngn.cuc", "rot13"), data={"page": page, "search": search})

            if response.status_code == 200:
                table_data = response.json().get("table", "")
                if not table_data.strip(): break
                all_table_data.append(table_data)
                page += 1
            else: raise Exception(f"An error occurred, Error code: {response.status_code}")
        except json.JSONDecodeError:
            raise Exception("An error occurred, unable to parse the response.")
        except requests.RequestException as e:
            raise Exception(f"Failed to send request: {e}")
    return all_table_data

def search_models(name):
    if not name: raise NameError("Please enter a model name")
    tables = fetch_models_data(name)

    if len(tables) == 0: print("No results found...")
    else:
        for table in tables:
            for row in BeautifulSoup(table, "html.parser").select("tr"):
                name_tag, url_tag = row.find("a", {"class": "fs-5"}), row.find("a", {"class": "btn btn-sm fw-bold btn-light ms-0 p-1 ps-2 pe-2"})
                if name_tag and url_tag:
                    name = name_tag.text.replace(".onnx", "").replace(".pth", "").replace(".index", "").replace(".zip", "").replace(" ", "_").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(",", "").replace('"', "").replace("'", "").replace("|", "_").replace("-_-", "_").replace("_-_", "_").replace("-", "_").replace("---", "_").replace("___", "_").strip()
                    url = url_tag["href"].replace("https://easyaivooice.com/run?url=", "")
                    if "huggingface" in url: print(f"{name}: {url}")

#@markdown **Search for model links**
model_name = "" # @param {"type":"string","placeholder":"Name of the model to search for"}

search_models(model_name)

In [ ]:
#@title **📩 Download Model**
from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

#@markdown **Supports links from huggingface.co / drive.google.com / mega.nz / mediafire.com**

model_name = "" # @param {"type":"string","placeholder":"Model name"}
model_link = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

if not model_link:
    uploaded = files.upload()
    args = f'from main.app.core.process import save_drop_model; save_drop_model(\\"{[list(uploaded.keys())[0]]}\\")'
    !python3 -c "$args"
else:
    args = f'from main.app.core.downloads import download_model; download_model(\\"{model_link}\\", \\"{model_name}\\")'
    !python3 -c "$args"

clear_output()
display(Button(description="\u2714 Completed!!", button_style="success"))

In [ ]:
#@title 🧾 **Download Pre-trained Models**
#@markdown **Run this cell to select a model and its sampling rate for download**
import os
import shutil
import codecs
import requests

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

from main.tools import huggingface

def fetch_pretrained_data():
    response = requests.get(codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/wfba/phfgbz_cergenvarq.wfba", "rot13"))
    response.raise_for_status()
    return response.json()

model_list = list(fetch_pretrained_data().keys())

for m in model_list:
    print(f"{model_list.index(m)}. {m}")

while 1:
    try:
        model_index = int(input("Enter the model number: "))
        if 0 <= model_index < len(model_list):
            selected_model = model_list[model_index]
            clear_output()
            print(f"You selected model: {selected_model}")
            break
        else: print("Invalid number. Please enter again.")
    except ValueError:
        print("Please enter an integer.")
    except IndexError:
        print("Number out of range. Please enter again.")

model_sr_list = list(fetch_pretrained_data()[selected_model].keys())

if len(model_sr_list) == 1: selected_sr = model_sr_list[0]
else:
    for sr in model_sr_list:
        print(f"{model_sr_list.index(sr)}. {sr}")

    while 1:
        try:
            model_sr_index = int(input("Enter the sampling rate number: "))
            if 0 <= model_sr_index < len(model_sr_list):
                selected_sr = model_sr_list[model_sr_index]
                print(f"You selected sampling rate: {selected_sr}")
                clear_output()
                break
            else: print("Invalid number. Please enter again.")
        except ValueError:
            print("Please enter an integer.")
        except IndexError:
            print("Number out of range. Please enter again.")

paths = fetch_pretrained_data()[selected_model][selected_sr]
pretraineds_custom_path = os.path.join("assets", "models", "pretrained_custom")

if not os.path.exists(pretraineds_custom_path): os.makedirs(pretraineds_custom_path, exist_ok=True)
url = codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/cergenvarq_phfgbz/", "rot13") + paths

file = huggingface.HF_download_file(url.replace("/blob/", "/resolve/").replace("?download=true", "").strip(), os.path.join(pretraineds_custom_path, paths))
if file.endswith(".zip"):
    shutil.unpack_archive(file, pretraineds_custom_path)
    os.remove(file)

clear_output()
Button(description="\u2714 Completed!", button_style="success")

In [ ]:
#@title 📥 **Download Pre-trained Models**
import os
import json

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/main_core

#@markdown **Enter the pre-trained model URLs to download**
pretrained_D = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}
pretrained_G = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

download_url = "Download from the link" if json.load(open(os.path.join("main", "configs", "config.json"), "r")).get("language", "vi-VN") == "en-US" else "Download from the link"

args = f'from main.app.core.downloads import download_pretrained_model; download_pretrained_model(\\"{download_url}\\", \\"{pretrained_D}\\", \\"{pretrained_G}\\")'
!python3 -c "$args"

clear_output()
Button(description="\u2714 Completed!", button_style="success")#@title 📥 **Download Pre-trained Models**
import os
import json

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

#@markdown **Enter the pre-trained model URLs to download**
pretrained_D = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}
pretrained_G = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

download_url = "Download from the link" if json.load(open(os.path.join("main", "configs", "config.json"), "r")).get("language", "vi-VN") == "en-US" else "Download from the link"

args = f'from main.app.core.downloads import download_pretrained_model; download_pretrained_model(\\"{download_url}\\", \\"{pretrained_D}\\", \\"{pretrained_G}\\")'
!python3 -c "$args"

clear_output()
Button(description="\u2714 Completed!", button_style="success")

# **Inference**

In [ ]:
#@title **Music Separation**
import os
import re
import yt_dlp
import shutil
import warnings

from ipywidgets import Button
from google.colab import files
from urllib.parse import urlparse
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

def download_url(url):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        ydl_opts = {"format": "bestaudio/best", "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav", "preferredquality": "192"}], "quiet": True, "no_warnings": True, "noplaylist": True, "verbose": False}

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output = os.path.join("audios", re.sub(r'\s+', '-', re.sub(r'[^\w\s\u4e00-\u9fff\uac00-\ud7af\u0400-\u04FF\u1100-\u11FF]', '', ydl.extract_info(url, download=False).get('title', 'video')).strip()))
            if os.path.exists(audio_output): shutil.rmtree(audio_output, ignore_errors=True)

            ydl_opts['outtmpl'] = audio_output

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output += ".wav"
            ydl.download([url])

        return audio_output

def is_url(path):
    try:
        result = urlparse(path)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

#@markdown **Options for separating stems and reverb**
separate_stems = False # @param {"type":"boolean"}
separate_reverb = False # @param {"type":"boolean"}
#@markdown **Input path (can be a URL or file path) and output audio format**
path = "" #@param {"type":"string", "placeholder":"Enter path to audio file or audio URL"}
audio_format = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Music separation model options, overlap level, and segment size**
separation_model = "Voc_FT" #@param ["Main_340", "Main_390", "Main_406", "Main_427", "Main_438", "Inst_full_292", "Inst_HQ_1", "Inst_HQ_2", "Inst_HQ_3", "Inst_HQ_4", "Inst_HQ_5", "Kim_Vocal_1", "Kim_Vocal_2", "Kim_Inst", "Inst_187_beta", "Inst_82_beta", "Inst_90_beta", "Voc_FT", "Crowd_HQ", "Inst_1", "Inst_2", "Inst_3", "MDXNET_1_9703", "MDXNET_2_9682", "MDXNET_3_9662", "Inst_Main", "MDXNET_Main", "MDXNET_9482", "HT-Normal", "HT-Tuned", "HD_MMI",  "HT_6S"]
overlap = "0.25" # @param ["0.25", "0.5", "0.75", "0.99"]
segment_size = 336 # @param {"type":"slider","min":32,"max":2048,"step":8}

if not path:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = download_url(path) if is_url(path) else path

args = f'from main.app.core.separate import separate_music; separate_music(\\"{input_path}\\", \\"audios\\", \\"{audio_format}\\", \\"{separation_model}\\", \\"MDX-Version-2\\", \\"MDX-Reverb\\", \\"Lite\\", 48000, 2, 1, {overlap}, 5, 1024, 512, {segment_size}, 0.2, False, True, False, False, {separate_stems}, {separate_reverb})'
!python3 -c "$args"

clear_output()
print(f"Your output files are located in: /content/Vietnamese_RVC/audios/{os.path.splitext(os.path.basename(input_path))[0]}")
display(Button(description="\u2714 Complete!", button_style="success"))

In [ ]:
#@title 🎵 **Audio Conversion** 🎵
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/main_core

def convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, input_audio_name, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode):
    args = f'from main.app.core.inference import convert_audio; convert_audio({clean}, {autotune}, {use_audio}, {use_original}, {convert_backing}, {not_merge_backing}, {merge_instrument}, {pitch}, {clean_strength}, \\"{model}\\", \\"{index}\\", {index_rate}, \\"{input}\\", \\"{output}\\", \\"{format}\\", \\"{method}\\", \\"{hybrid_method}\\", {hop_length}, \\"{embedders}\\", \\"{custom_embedders}\\", {resample_sr}, {filter_radius}, {volume_envelope}, {protect}, {split_audio}, {f0_autotune_strength}, \\"{input_audio_name}\\", {checkpointing}, {onnx_f0_mode}, {formant_shifting}, {formant_qfrency}, {formant_timbre}, \\"{f0_file}\\", \\"{embedders_mode}\\", False, 0)'
    !python3 -c "$args"

def get_audio_file(output_audio, label):
    matching_files = [f for f in os.listdir(output_audio) if label in f]

    if not matching_files: raise FileNotFoundError("No backing track found!")
    return os.path.join(output_audio, matching_files[0])

def convert_selection(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode):
    if use_audio:
        choice = [f for f in os.listdir("audios") if os.path.isdir(os.path.join("audios", f))]

        if len(choice) == 0: raise FileNotFoundError("No separated tracks found!")
        elif len(choice) == 1:
            choice_audio = choice[0]
            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode)
        else:
            print("Found more than 1 separated track, please select one to proceed with conversion!")
            for c in choice:
                print(f"{choice.index(c)}. {c}")

            while 1:
                try:
                    choice_select = int(input("Enter the number of the separated track: "))

                    if 0 <= choice_select < len(choice):
                        choice_audio = choice[choice_select]
                        print(f"You selected the track: {choice_audio}")
                        break
                    else: print("Invalid number. Please try again.")
                except ValueError:
                    print("Please enter an integer.")
                except IndexError:
                    print("Number out of range. Please try again.")
            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode)
    else: convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, None, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode)

    if use_audio: output_path = f"/content/Vietnamese_RVC/audios/{choice_audio}/Vocals+Backing.{format}" if convert_backing else f"/content/Vietnamese_RVC/audios/{choice_audio}/Convert_Vocals.{format}"
    return audio_effects(output_path, os.path.join("audios", choice_audio, f"Vocals+Instruments.{format}") if merge_instrument else os.path.join("audios", choice_audio, f"{os.path.splitext(os.path.basename(output_path))[0]}_Effects.{format}"), format, merge_instrument, get_audio_file(os.path.join("audios", choice_audio), "Instruments.")) if use_audio else f"/content/Vietnamese_RVC/audios/output.{format}"

def audio_effects(input_path, output_path, export_format, merge_instrument, audio_combination_input):
    if not input_path or not os.path.exists(input_path) or os.path.isdir(input_path): raise FileNotFoundError("Invalid input file path!")
    if not output_path: raise ValueError("Invalid output file path!")
    output_dir = os.path.dirname(output_path) or output_path

    if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)
    if os.path.exists(output_path): os.remove(output_path)

    !python main/inference/audio_effects.py --input_path $input_path --output_path $output_path --reverb_room_size 0.15 --reverb_damping 0.7 --reverb_wet_level 0.2 --reverb_dry_level 0.8 --reverb_width 1.0 --reverb_freeze_mode False --export_format $export_format --reverb True --audio_combination $merge_instrument --audio_combination_input $audio_combination_input
    return output_path

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

#@markdown **Options for using previously separated audio tracks**
use_separated_audio = False #@param {"type":"boolean"}
convert_backing_track = False #@param {"type":"boolean"}
merge_with_instrumental = False #@param {"type":"boolean"}
#@markdown **Model options: Pitch and Index**
model_name = "" #@param {"type":"string","placeholder":"Model name"}
pitch_shift = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
index_influence = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Input file and output format options**
input_file_path = "" #@param {"type":"string","placeholder":"Audio file path"}
output_file_format = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Feature extraction options, hop length, voice protection, and automatic pitch adjustment for higher quality**
extraction_method = "rmvpe" #@param ["pm-ac", "pm-cc", "pm-shs", "dio", "mangio-crepe-tiny", "mangio-crepe-small", "mangio-crepe-medium", "mangio-crepe-large", "mangio-crepe-full", "crepe-tiny", "crepe-small", "crepe-medium", "crepe-large", "crepe-full", "fcpe", "fcpe-legacy", "rmvpe", "rmvpe-legacy", "harvest", "yin", "pyin", "swipe", "piptrack", "fcn", "djcm"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
voice_protection = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
#@markdown **Audio splitting and automatic pitch options**
split_audio = False #@param {"type":"boolean"}

if model_name:
    if not model_name.endswith((".pth", ".onnx")): model_name = model_name + ".pth"
    model_path = os.path.join("assets", "weights", model_name)
    index_path = get_index(model_name.split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Please provide a model to proceed with conversion!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Found more than 1 model, please enter the model number to proceed with conversion!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Enter the model number: "))
                if model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"You selected the model: {selected_model}")
                    break
                else: print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter an integer.")
            except IndexError:
                print("Number out of range. Please try again.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("No index found!")

if not input_file_path and not (use_separated_audio or convert_backing_track or merge_with_instrumental):
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = input_file_path

output_files = convert_selection(False, False, (use_separated_audio or convert_backing_track or merge_with_instrumental), use_separated_audio, convert_backing_track, False, merge_with_instrumental, pitch_shift, 0.7, model_path, index_path, index_influence, input_path, os.path.join("audios", "output.wav"), output_file_format, extraction_method, None, hop_length, "hubert_base", None, 0, 3, 1, voice_protection, split_audio, 1, False, False, False, 0, 0, "", "fairseq")
clear_output()

print(f"Your output file is: {output_files}")
display(Button(description="\u2714 Complete!", button_style="success"))

In [ ]:

#@title 🎵 **Text-to-Speech Conversion** 🎵
import os
import zipfile
import xml.etree.ElementTree

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/main_core

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

def read_docx_text(path):
    with zipfile.ZipFile(path) as docx:
        with docx.open("word/document.xml") as document_xml:
            xml_content = document_xml.read()

    WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'

    paragraphs = []
    for paragraph in xml.etree.ElementTree.XML(xml_content).iter(WORD_NAMESPACE + 'p'):
        texts = [node.text for node in paragraph.iter(WORD_NAMESPACE + 't') if node.text]
        if texts: paragraphs.append(''.join(texts))

    return '\n'.join(paragraphs)

def process_input(file_path):
    if file_path.endswith(".srt"): file_contents = ""
    elif file_path.endswith(".docx"): file_contents = read_docx_text(file_path)
    else:
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                file_contents = file.read()
        except Exception as e:
            file_contents = ""

    return file_contents

#@markdown **Text to read and voice/speech rate options**
text_input = "" # @param {"type":"string", "placeholder":"Text to read"}
voice = "jv-ID-DimasNeural" # @param ["af-ZA-AdriNeural", "af-ZA-WillemNeural", "sq-AL-AnilaNeural", "sq-AL-IlirNeural", "am-ET-AmehaNeural", "am-ET-MekdesNeural", "ar-DZ-AminaNeural", "ar-DZ-IsmaelNeural", "ar-BH-AliNeural", "ar-BH-LailaNeural", "ar-EG-SalmaNeural", "ar-EG-ShakirNeural", "ar-IQ-BasselNeural", "ar-IQ-RanaNeural", "ar-JO-SanaNeural", "ar-JO-TaimNeural", "ar-KW-FahedNeural", "ar-KW-NouraNeural", "ar-LB-LaylaNeural", "ar-LB-RamiNeural", "ar-LY-ImanNeural", "ar-LY-OmarNeural", "ar-MA-JamalNeural", "ar-MA-MounaNeural", "ar-OM-AbdullahNeural", "ar-OM-AyshaNeural", "ar-QA-AmalNeural", "ar-QA-MoazNeural", "ar-SA-HamedNeural", "ar-SA-ZariyahNeural", "ar-SY-AmanyNeural", "ar-SY-LaithNeural", "ar-TN-HediNeural", "ar-TN-ReemNeural", "ar-AE-FatimaNeural", "ar-AE-HamdanNeural", "ar-YE-MaryamNeural", "ar-YE-SalehNeural", "az-AZ-BabekNeural", "az-AZ-BanuNeural", "bn-BD-NabanitaNeural", "bn-BD-PradeepNeural", "bn-IN-BashkarNeural", "bn-IN-TanishaaNeural", "bs-BA-GoranNeural", "bs-BA-VesnaNeural", "bg-BG-BorislavNeural", "bg-BG-KalinaNeural", "my-MM-NilarNeural", "my-MM-ThihaNeural", "ca-ES-EnricNeural", "ca-ES-JoanaNeural", "zh-HK-HiuGaaiNeural", "zh-HK-HiuMaanNeural", "zh-HK-WanLungNeural", "zh-CN-XiaoxiaoNeural", "zh-CN-XiaoyiNeural", "zh-CN-YunjianNeural", "zh-CN-YunxiNeural", "zh-CN-YunxiaNeural", "zh-CN-YunyangNeural", "zh-CN-liaoning-XiaobeiNeural", "zh-TW-HsiaoChenNeural", "zh-TW-YunJheNeural", "zh-TW-HsiaoYuNeural", "zh-CN-shaanxi-XiaoniNeural", "hr-HR-GabrijelaNeural", "hr-HR-SreckoNeural", "cs-CZ-AntoninNeural", "cs-CZ-VlastaNeural", "da-DK-ChristelNeural", "da-DK-JeppeNeural", "nl-BE-ArnaudNeural", "nl-BE-DenaNeural", "nl-NL-ColetteNeural", "nl-NL-FennaNeural", "nl-NL-MaartenNeural", "en-AU-NatashaNeural", "en-AU-WilliamNeural", "en-CA-ClaraNeural", "en-CA-LiamNeural", "en-HK-SamNeural", "en-HK-YanNeural", "en-IN-NeerjaExpressiveNeural", "en-IN-NeerjaNeural", "en-IN-PrabhatNeural", "en-IE-ConnorNeural", "en-IE-EmilyNeural", "en-KE-AsiliaNeural", "en-KE-ChilembaNeural", "en-NZ-MitchellNeural", "en-NZ-MollyNeural", "en-NG-AbeoNeural", "en-NG-EzinneNeural", "en-PH-JamesNeural", "en-PH-RosaNeural", "en-SG-LunaNeural", "en-SG-WayneNeural", "en-ZA-LeahNeural", "en-ZA-LukeNeural", "en-TZ-ElimuNeural", "en-TZ-ImaniNeural", "en-GB-LibbyNeural", "en-GB-MaisieNeural", "en-GB-RyanNeural", "en-GB-SoniaNeural", "en-GB-ThomasNeural", "en-US-AvaMultilingualNeural", "en-US-AndrewMultilingualNeural", "en-US-EmmaMultilingualNeural", "en-US-BrianMultilingualNeural", "en-US-AvaNeural", "en-US-AndrewNeural", "en-US-EmmaNeural", "en-US-BrianNeural", "en-US-AnaNeural", "en-US-AriaNeural", "en-US-ChristopherNeural", "en-US-EricNeural", "en-US-GuyNeural", "en-US-JennyNeural", "en-US-MichelleNeural", "en-US-RogerNeural", "en-US-SteffanNeural", "et-EE-AnuNeural", "et-EE-KertNeural", "fil-PH-AngeloNeural", "fil-PH-BlessicaNeural", "fi-FI-HarriNeural", "fi-FI-NooraNeural", "fr-BE-CharlineNeural", "fr-BE-GerardNeural", "fr-CA-ThierryNeural", "fr-CA-AntoineNeural", "fr-CA-JeanNeural", "fr-CA-SylvieNeural", "fr-FR-VivienneMultilingualNeural", "fr-FR-RemyMultilingualNeural", "fr-FR-DeniseNeural", "fr-FR-EloiseNeural", "fr-FR-HenriNeural", "fr-CH-ArianeNeural", "fr-CH-FabriceNeural", "gl-ES-RoiNeural", "gl-ES-SabelaNeural", "ka-GE-EkaNeural", "ka-GE-GiorgiNeural", "de-AT-IngridNeural", "de-AT-JonasNeural", "de-DE-SeraphinaMultilingualNeural", "de-DE-FlorianMultilingualNeural", "de-DE-AmalaNeural", "de-DE-ConradNeural", "de-DE-KatjaNeural", "de-DE-KillianNeural", "de-CH-JanNeural", "de-CH-LeniNeural", "el-GR-AthinaNeural", "el-GR-NestorasNeural", "gu-IN-DhwaniNeural", "gu-IN-NiranjanNeural", "he-IL-AvriNeural", "he-IL-HilaNeural", "hi-IN-MadhurNeural", "hi-IN-SwaraNeural", "hu-HU-NoemiNeural", "hu-HU-TamasNeural", "is-IS-GudrunNeural", "is-IS-GunnarNeural", "id-ID-ArdiNeural", "id-ID-GadisNeural", "ga-IE-ColmNeural", "ga-IE-OrlaNeural", "it-IT-GiuseppeNeural", "it-IT-DiegoNeural", "it-IT-ElsaNeural", "it-IT-IsabellaNeural", "ja-JP-KeitaNeural", "ja-JP-NanamiNeural", "jv-ID-DimasNeural", "jv-ID-SitiNeural", "kn-IN-GaganNeural", "kn-IN-SapnaNeural", "kk-KZ-AigulNeural", "kk-KZ-DauletNeural", "km-KH-PisethNeural", "km-KH-SreymomNeural", "ko-KR-HyunsuNeural", "ko-KR-InJoonNeural", "ko-KR-SunHiNeural", "lo-LA-ChanthavongNeural", "lo-LA-KeomanyNeural", "lv-LV-EveritaNeural", "lv-LV-NilsNeural", "lt-LT-LeonasNeural", "lt-LT-OnaNeural", "mk-MK-AleksandarNeural", "mk-MK-MarijaNeural", "ms-MY-OsmanNeural", "ms-MY-YasminNeural", "ml-IN-MidhunNeural", "ml-IN-SobhanaNeural", "mt-MT-GraceNeural", "mt-MT-JosephNeural", "mr-IN-AarohiNeural", "mr-IN-ManoharNeural", "mn-MN-BataaNeural", "mn-MN-YesuiNeural", "ne-NP-HemkalaNeural", "ne-NP-SagarNeural", "nb-NO-FinnNeural", "nb-NO-PernilleNeural", "ps-AF-GulNawazNeural", "ps-AF-LatifaNeural", "fa-IR-DilaraNeural", "fa-IR-FaridNeural", "pl-PL-MarekNeural", "pl-PL-ZofiaNeural", "pt-BR-ThalitaNeural", "pt-BR-AntonioNeural", "pt-BR-FranciscaNeural", "pt-PT-DuarteNeural", "pt-PT-RaquelNeural", "ro-RO-AlinaNeural", "ro-RO-EmilNeural", "ru-RU-DmitryNeural", "ru-RU-SvetlanaNeural", "sr-RS-NicholasNeural", "sr-RS-SophieNeural", "si-LK-SameeraNeural", "si-LK-ThiliniNeural", "sk-SK-LukasNeural", "sk-SK-ViktoriaNeural", "sl-SI-PetraNeural", "sl-SI-RokNeural", "so-SO-MuuseNeural", "so-SO-UbaxNeural", "es-AR-ElenaNeural", "es-AR-TomasNeural", "es-BO-MarceloNeural", "es-BO-SofiaNeural", "es-CL-CatalinaNeural", "es-CL-LorenzoNeural", "es-ES-XimenaNeural", "es-CO-GonzaloNeural", "es-CO-SalomeNeural", "es-CR-JuanNeural", "es-CR-MariaNeural", "es-CU-BelkysNeural", "es-CU-ManuelNeural", "es-DO-EmilioNeural", "es-DO-RamonaNeural", "es-EC-AndreaNeural", "es-EC-LuisNeural", "es-SV-LorenaNeural", "es-SV-RodrigoNeural", "es-GQ-JavierNeural", "es-GQ-TeresaNeural", "es-GT-AndresNeural", "es-GT-MartaNeural", "es-HN-CarlosNeural", "es-HN-KarlaNeural", "es-MX-DaliaNeural", "es-MX-JorgeNeural", "es-NI-FedericoNeural", "es-NI-YolandaNeural", "es-PA-MargaritaNeural", "es-PA-RobertoNeural", "es-PY-MarioNeural", "es-PY-TaniaNeural", "es-PE-AlexNeural", "es-PE-CamilaNeural", "es-PR-KarinaNeural", "es-PR-VictorNeural", "es-ES-AlvaroNeural", "es-ES-ElviraNeural", "es-US-AlonsoNeural", "es-US-PalomaNeural", "es-UY-MateoNeural", "es-UY-ValentinaNeural", "es-VE-PaolaNeural", "es-VE-SebastianNeural", "su-ID-JajangNeural", "su-ID-TutiNeural", "sw-KE-RafikiNeural", "sw-KE-ZuriNeural", "sw-TZ-DaudiNeural", "sw-TZ-RehemaNeural", "sv-SE-MattiasNeural", "sv-SE-SofieNeural", "ta-IN-PallaviNeural", "ta-IN-ValluvarNeural", "ta-MY-KaniNeural", "ta-MY-SuryaNeural", "ta-SG-AnbuNeural", "ta-SG-VenbaNeural", "ta-LK-KumarNeural", "ta-LK-SaranyaNeural", "te-IN-MohanNeural", "te-IN-ShrutiNeural", "th-TH-NiwatNeural", "th-TH-PremwadeeNeural", "tr-TR-AhmetNeural", "tr-TR-EmelNeural", "uk-UA-OstapNeural", "uk-UA-PolinaNeural", "ur-IN-GulNeural", "ur-IN-SalmanNeural", "ur-PK-AsadNeural", "ur-PK-UzmaNeural", "uz-UZ-MadinaNeural", "uz-UZ-SardorNeural", "vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural", "cy-GB-AledNeural", "cy-GB-NiaNeural", "zu-ZA-ThandoNeural", "zu-ZA-ThembaNeural"]
speech_rate = 0 # @param {"type":"slider","min":-100,"max":100,"step":1}
#@markdown **Model options: Pitch and Index**
model_name = "" #@param {"type":"string","placeholder":"Model name"}
pitch_shift = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
index_influence = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Feature extraction options, hop length, voice protection**
extraction_method = "rmvpe" #@param ["pm-ac", "pm-cc", "pm-shs", "dio", "mangio-crepe-tiny", "mangio-crepe-small", "mangio-crepe-medium", "mangio-crepe-large", "mangio-crepe-full", "crepe-tiny", "crepe-small", "crepe-medium", "crepe-large", "crepe-full", "fcpe", "fcpe-legacy", "rmvpe", "rmvpe-legacy", "harvest", "yin", "pyin", "swipe", "piptrack", "fcn"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
voice_protection = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
#@markdown **Options for audio splitting for faster processing, automatic pitch, and output file format**
output_file_format = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
split_audio = False #@param {"type":"boolean"}

filename = None
if text_input: input_text = text_input
else:
    print("Text field is empty. Please upload a text file (.txt) to proceed with conversion!")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_text = process_input(filename)

if model_name:
    if not model_name.endswith((".pth", ".onnx")): model_name = model_name + ".pth"
    model_path = os.path.join("assets", "weights", model_name)
    index_path = get_index(f"{model_name}.pth".split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Please provide a model to proceed with conversion!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Found more than 1 model, please enter the model number to proceed with conversion!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Enter the model number: "))
                if 0 <= model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"You selected the model: {selected_model}")
                    break
                else: print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter an integer.")
            except IndexError:
                print("Number out of range. Please try again.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("No index found!")

tts_output = os.path.join("audios", f"tts.{output_file_format}")
convert_tts_output = os.path.join("audios", f"tts-convert.{output_file_format}")

args = f'from main.app.core.tts import TTS; TTS(\\"{input_text}\\", \\"{voice}\\", {speech_rate}, \\"{tts_output}\\", 0, False, \\"{filename}\\")'
!python3 -c "$args"

args = f'from main.app.core.inference import convert_tts; convert_tts(False, False, {pitch_shift}, 0.7, \\"{model_path}\\", \\"{index_path}\\", {index_influence}, \\"{tts_output}\\", \\"{convert_tts_output}\\", \\"{output_file_format}\\", \\"{extraction_method}\\", None, {hop_length}, \\"hubert_base\\", None, 0, 3, 1, {voice_protection}, {split_audio}, 1, False, False, False, 0, 0, \\"None\\", \\"fairseq\\", False, 0)'
!python3 -c "$args"

clear_output()
print(f"Your output file is: /content/Vietnamese_RVC/audios/tts-convert.{output_file_format}")
display(Button(description="\u2714 Complete!", button_style="success"))

# **Model Training 🤖**

In [ ]:
#@title 📁 **Create training data from path**
from ipywidgets import Button
from IPython.display import clear_output, display

%cd /content/main_core

#@markdown **Create training data from URL**
clean_data = False #@param {"type":"boolean"}
url_path = "" #@param {"type":"string", placeholder:"URL path"}

args = f'from main.app.core.training import create_dataset; [None for _ in create_dataset(\\"{url_path}\\", \\"dataset\\", False, 0, 0, True, \\"HD_MMI\\", \\"MDX-Reverb\\", \\"Lite\\", 48000, 2, 1, 0.25, 5, 1024, 512, 256, 0.2, False, True, False, False, True, {clean_data}, 0.7)]'
!python3 -c "$args"

clear_output()
display(Button(description="\u2714 Complete!", button_style="success"))

In [ ]:

#@title 🔨 **Data Preprocessing & Feature Extraction** ⛏️
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/main_core

#@markdown **Load data from Google Drive to Google Colab for processing and extraction**
load_from_drive = False # @param {"type":"boolean"}
#@markdown **Model name for processing**
model_name = "" #@param {"type":"string", placeholder: "Model name"}
#@markdown **Model sampling rate for processing**
sampling_rate = "48k" #@param ["32k", "40k", "48k"]
#@markdown **Pitch extraction method and hop length for the model**
extraction_method = "rmvpe" #@param ["pm-ac", "pm-cc", "pm-shs", "dio", "mangio-crepe-tiny", "mangio-crepe-small", "mangio-crepe-medium", "mangio-crepe-large", "mangio-crepe-full", "crepe-tiny", "crepe-small", "crepe-medium", "crepe-large", "crepe-full", "fcpe", "fcpe-legacy", "rmvpe", "rmvpe-legacy", "harvest", "yin", "pyin", "swipe", "piptrack", "fcn"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}

if load_from_drive:
    if not os.path.exists("/content/drive/MyDrive"): raise ValueError("You haven't connected to Google Drive")
    if len([f for f in os.listdir("/content/drive/MyDrive/dataset") if not ".ipynb_checkpoints" in f]) < 1: raise FileNotFoundError("No data found")

    for audios in os.listdir("/content/drive/MyDrive/dataset"):
        shutil.copy(f"/content/drive/MyDrive/dataset/{audios}", "/content/Vietnamese_RVC/dataset")
elif not any(f.lower().endswith(("wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3")) for f in os.listdir("/content/Vietnamese_RVC/dataset") if os.path.isfile(os.path.join("/content/Vietnamese_RVC/dataset", f))):
    uploaded = files.upload()
    for f in list(uploaded.keys()):
        input_path = os.path.join("dataset", os.path.basename(f).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
        shutil.move(f, input_path)

if not model_name: raise ValueError("Please provide a name to proceed!")

args = f'from main.app.core.training import preprocess; [None for _ in preprocess(\\"{model_name}\\", \\"{sampling_rate}\\", 2, True, True, \\"dataset\\", False, 0.7)]'
!python3 -c "$args"

args = f'from main.app.core.training import extract; [None for _ in extract(\\"{model_name}\\", \\"v2\\", \\"{extraction_method}\\", True, {hop_length}, 2, 0, \\"{sampling_rate}\\", \\"hubert_base\\", None, False, \\"fairseq\\", False, 1, None, False)]'
!python3 -c "$args"

args = f'from main.app.core.training import create_index; [None for _ in create_index(\\"{model_name}\\", \\"v2\\", \\"Auto\\")]'
!python3 -c "$args"

clear_output()
display(Button(description="\u2714 Complete!", button_style="success"))

In [ ]:

#@title 🤖 **Model Training**
import os

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/main_core

#@markdown **Model name and sampling rate for training**
model_name = "" #@param {"type":"string", placeholder: "Model name"}
sampling_rate = "48k" #@param ["32k", "40k", "48k"]
#@markdown **Number of epochs and save frequency**
num_epochs = 300 #@param {"type":"slider", min:1, max:10000}
save_frequency = 50 #@param {"type":"slider", min:1, max:10000}
#@markdown **Batch size options and whether to use cache**
batch_size = 10 #@param {"type":"slider", min:1, max:20}
use_cache = False #@param {"type":"boolean"}
#@markdown **Training visualization options, automatic overfitting check, and using custom pre-trained models**
use_tensorboard = False #@param {type:"boolean"}
check_overfitting = False #@param {"type":"boolean"}
use_custom_pretrained = False #@param {"type":"boolean"}

if use_tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

if use_custom_pretrained:
    model_name = [f for f in os.listdir(os.path.join("assets", "models", "pretrained_custom")) if f.endswith(".pth")]

    if len(model_name) == 2:
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index_d = int(input("Enter the number for model D: "))
                if 0 <= model_index_d < len(model_name):
                    selected_model_D = model_name[model_index_d]
                    print(f"You selected model D: {selected_model_D}")
                    break
                else: print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter an integer.")
            except IndexError:
                print("Number out of range. Please try again.")

        model_name.remove(selected_model_D)
        model_index_g = model_name[0]
    elif len(model_name) >= 2:
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index_d = int(input("Enter the number for model D: "))
                if 0 <= model_index_d < len(model_name):
                    selected_model_D = model_name[model_index_d]
                    print(f"You selected model D: {selected_model_D}")
                    break
                else: print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter an integer.")
            except IndexError:
                print("Number out of range. Please try again.")

        while 1:
            try:
                model_index_g = int(input("Enter the number for model G: "))
                if 0 <= model_index_g < len(model_name):
                    selected_model_G = model_name[model_index_g]
                    print(f"You selected model G: {selected_model_G}")
                    break
                else: print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter an integer.")
            except IndexError:
                print("Number out of range. Please try again.")

        if selected_model_D == selected_model_G: raise ValueError("Model D and G are the same!")
        clear_output()
    else: print("No pre-trained models found!")
else: selected_model_G = selected_model_D = None


args = f'from main.app.core.training import training; [None for _ in training(\\"{model_name}\\", \\"v2\\", {save_frequency}, True, True, {num_epochs}, \\"{sampling_rate}\\", {batch_size}, 0, True, False, {use_custom_pretrained}, \\"{selected_model_G}\\", \\"{selected_model_D}\\", {check_overfitting}, 50, False, {use_cache}, \\"\\", \\"Default\\", False, False, True, \\"AdamW\\", False)]'
!python3 -c "$args"

Button(description="\u2714 Complete!", button_style="success")

In [ ]:

#@title 📦 **Compress and Save Model** 📦

#@markdown **Run this cell and select the model file to compress!**

import os
import zipfile

from ipywidgets import Button
from IPython.display import clear_output

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

if len(model_name) < 1: raise ValueError("Please provide a model to proceed!")
elif len(model_name) == 1:
    model_path = os.path.join("assets", "weights", model_name[0])
    index_path = get_index(os.path.basename(model_name[0])[0])
else:
    print("Found more than 1 model, please enter the model number to proceed!")
    for m in model_name:
        print(f"{model_name.index(m)}. {m}")

    while 1:
        try:
            model_index = int(input("Enter the model number: "))
            if 0 <= model_index < len(model_name):
                selected_model = model_name[model_index]
                print(f"You selected the model: {selected_model}")
                break
            else: print("Invalid number. Please try again.")
        except ValueError:
            print("Please enter an integer.")
        except IndexError:
            print("Number out of range. Please try again.")

    model_path = os.path.join("assets", "weights", selected_model)
    index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("No index found!")
zip_file_path = os.path.join("assets", os.path.basename(model_path).split("_")[0] + ".zip")

with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(model_path, os.path.basename(model_path))
    if index_path: zipf.write(index_path, os.path.basename(index_path))

clear_output()
print(f"Your model path: {zip_file_path}")
Button(description="\u2714 Complete!", button_style="success")